In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [15]:
top = 100
# split_pair_path = "../data/split_dev/split_pair_dev_k_{0}.txt".format(top)
# train_data_path = "../data/split_dev/train_set.txt"
# dev_data_path = "../data/split_dev/dev_set_v1_4.txt"

split_pair_path = "../data/encode_3epoch/split_pair_dev_k_{0}.txt".format(top)
train_data_path = "../data/encode_3epoch/train_set.txt"
dev_data_path = "../data/encode_3epoch/dev_set_v1_4.txt"


train_data = pd.read_csv(train_data_path, sep='\t', header=None)
train_label = np.array(train_data.iloc[:,0]).astype('int')
dev_data = pd.read_csv(dev_data_path, sep='\t', header=None)
dev_label = np.array(dev_data.iloc[:,0]).astype('int')
split_data = pd.read_csv(split_pair_path, sep='\t')

In [16]:
split_yes_no_list = np.array(split_data.iloc[:,2])
print("positive:negative = {0}:{1}".format(sum(split_yes_no_list==1), sum(split_yes_no_list==0)))

positive:negative = 178229:3104471


In [17]:
%%time

total_single_split_num = int(len(split_yes_no_list)/top)
counter = 0
index_list = []
for i in range(0, len(split_yes_no_list), top):
    single_split = split_yes_no_list[i:(i+top)]
    if 1 in single_split:
        #new_train_data = new_train_data.append(split_data.iloc[i:(i+top),:])
        index_list.append(i)
        counter += 1
print("recall: {0}/{1}={2:.4f}".format(counter, total_single_split_num, (counter/total_single_split_num)))

recall: 16601/32827=0.5057
CPU times: user 83.4 ms, sys: 75 µs, total: 83.5 ms
Wall time: 82.3 ms


In [1]:
%%time

new_train_data = pd.DataFrame()
for i in range(0, len(split_yes_no_list), top):
    single_split = split_yes_no_list[i:(i+top)]
    if 1 in single_split:
        new_train_data = new_train_data.append(split_data.iloc[i:(i+top),:])

NameError: name 'pd' is not defined

In [6]:
ratio = 1/ (sum(new_train_data.iloc[:,2]==1)/sum(new_train_data.iloc[:,2]==0))
print("positive:negative = {0}:{1}".format(sum(new_train_data.iloc[:,2]==1), sum(new_train_data.iloc[:,2]==0)))
print("ratio: {0}".format(1/ratio))
print("ratio: {0}".format(ratio))

positive:negative = 177881:1479519
ratio: 0.12022893927012768
ratio: 8.317465046857167


In [7]:
split_pair_path = "../data/split_dev/new_split_pair_dev_k_{0}.txt".format(top)

new_train_data.to_csv(split_pair_path, sep='\t', index=False)
#new_train_data = pd.read_csv(split_pair_path, sep='\t')
new_train_data = new_train_data.reset_index(drop=True)

In [15]:
k = 100
split_pair_path = "../data/first_filter_2epoch/new_split_pair_dev_2epoch_k_{0}.txt".format(k)

new_train_data = pd.read_csv(split_pair_path, sep='\t')

In [16]:
%%time
total_len = len(new_train_data)
unique_len = int(total_len/k)
original_shape = (unique_len, k)
true_sentence = np.array(new_train_data.true_sentence).reshape(original_shape)
candidate_sentence = np.array(new_train_data.candidate_sentence).reshape(original_shape)
pair_result = np.array(new_train_data.pair_result).reshape(original_shape)
true_label = np.array(new_train_data.true_label).reshape(original_shape)
candidate_label = np.array(new_train_data.candidate_label).reshape(original_shape)

CPU times: user 126 ms, sys: 12 ms, total: 138 ms
Wall time: 90.5 ms


In [17]:
# from probability file, get top k candidates
probability_file_path = "../data/first_filter_2epoch/test_results_k_{0}_of_k_1000.csv".format(k)

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]
probability_mat = np.array(probability_file).reshape(original_shape)

In [18]:
%%time
sorted_candidate_label = np.zeros((original_shape)).astype('int')
sorted_probability_mat = np.zeros((original_shape))
for i in range(unique_len):
    sorted_index = probability_mat[i].argsort()[::-1]
    sorted_candidate_label[i] = candidate_label[i, sorted_index]
    sorted_probability_mat[i] = probability_mat[i, sorted_index] 

CPU times: user 122 ms, sys: 11.9 ms, total: 134 ms
Wall time: 134 ms


In [53]:
sorted_candidate_label

array([[ 503,  503,  503, ..., 7129, 5393, 5492],
       [9796, 5245, 1887, ..., 6481, 9431, 9012],
       [ 262,  262,  262, ...,  829, 4120, 5232],
       ...,
       [5098, 5098, 5098, ...,  933, 5098, 7661],
       [9386, 9838, 9046, ..., 8673, 7717, 7785],
       [9887, 8974, 3550, ..., 9005,  934, 7639]])

In [54]:
sorted_probability_mat

array([[0.6988538 , 0.50782937, 0.2562292 , ..., 0.0177156 , 0.016883  ,
        0.01153505],
       [0.23331267, 0.2290624 , 0.10045436, ..., 0.01757839, 0.01288076,
        0.01103879],
       [0.9986482 , 0.9984952 , 0.997982  , ..., 0.02205376, 0.01964841,
        0.01539265],
       ...,
       [0.9987276 , 0.99706465, 0.99697363, ..., 0.01225922, 0.01073612,
        0.00469577],
       [0.78372437, 0.1429009 , 0.09616429, ..., 0.01279237, 0.01253942,
        0.0110232 ],
       [0.2605647 , 0.2118981 , 0.18246606, ..., 0.00762148, 0.00685781,
        0.00669714]])

In [21]:
def getFilteredLabel(weights, label_list):
    unique_label = np.unique(label_list)
    final_weights = np.zeros((unique_label.shape))
    for i, label in enumerate(unique_label):
        label_index = (label == label_list)
        final_weights[i] = sum(weights[label_index])
    sorted_index = final_weights.argsort()[::-1]
    return unique_label[sorted_index[0]]

def getWeightedLabel(weights, labels, threshold, filter=True):
    if filter:
        if weights[0] <= threshold:
            return labels[0]
        filtered_index = (weights > threshold)
        return getFilteredLabel(weights[filtered_index], labels[filtered_index])
    else:
        return getFilteredLabel(weights, labels)

def computeAccuracy(predicted, true_label):
    accuracy = accuracy_score(true_label, predicted)
    print("accuracy: {0}".format(accuracy))

In [52]:
%%time
top = 100
probability_threshold = 0.99
new_candidate_list = np.zeros((unique_len, 1)).astype('int')
for i in range(unique_len):
    new_candidate_list[i] = \
            getWeightedLabel(
                    sorted_probability_mat[i,:top],
                    sorted_candidate_label[i,:top],
                    probability_threshold)
computeAccuracy(new_candidate_list, true_label[:,0])

accuracy: 0.5643318167389694
CPU times: user 159 ms, sys: 3.95 ms, total: 163 ms
Wall time: 163 ms


In [ ]:
top = k
new_candidate_list = [None] * int(len(new_train_data)/top)
counter = 0
for i in range(0, len(new_train_data), top):
    new_train_data_single_split = new_train_data[i:(i+top)]
    new_train_label_single_split = np.array(new_train_data_single_split.candidate_label)
    probability_single_split = probability_file[i:(i+top)]
    sorted_index = np.array(np.argsort(probability_single_split)[::-1][:new_top])
    sorted_probability_split = np.array(probability_single_split)[sorted_index]
    new_train_label_single_split = new_train_label_single_split[sorted_index]
    filted_probability_split = sorted_probability_split[sorted_probability_split > probability_threshold]
    candidate_label = np.array([new_train_label_single_split[0]])
    if 0 != len(filted_probability_split):
        candidate_label = new_train_label_single_split[:len(filted_probability_split)]
    new_candidate_list[counter] = candidate_label
    counter += 1
   

In [12]:
new_top_true_label_list = np.zeros((len(new_candidate_list), 1))
new_top_predict_label_list = np.zeros((len(new_candidate_list), 1))
counter = 0
for i in range(0, len(new_candidate_list)):
    new_top_true_label_list[i] = new_train_data.true_label[i*top]
    predict_label = np.argmax(np.bincount(new_candidate_list[counter]))
    new_top_predict_label_list[i] = predict_label
    counter += 1

In [13]:
new_top_true_label_list = np.array(new_top_true_label_list)
accuracy = accuracy_score(new_top_true_label_list, new_top_predict_label_list)
print("accuracy: {0}".format(accuracy))

accuracy: 0.5623743519204317


In [16]:
new_top_true_label_list = np.array(new_top_true_label_list)
accuracy = accuracy_score(new_top_true_label_list, new_top_predict_label_list)
print("accuracy: {0}".format(accuracy))

accuracy: 0.674908050674295


dev 74367/582173=0.127

split_num = 1
recall: 15795/36746
positive sample: 95259/734920  = 0.1296
negative sample: 639661/734920 = 0.8703

split_num = 2
recall:  /36746

split_num = 3
recall: 36745/36746
positive sample: 94218/734920  = 0.1282
negative sample: 640702/734920 = 0.8717

split_num = 4
recall: 36744/36746
positive sample: 94365/734920  = 0.1284
negative sample: 640555/734920 = 0.8715

split_num = 5
recall: 36744/36746
positive sample: 94157/734920  = 0.1281
negative sample: 640763/734920 = 0.8718

split_num = 6
recall: 36744/36746
positive sample: 93945/734920  = 0.1278
negative sample: 640975/734920 = 0.8721

split_num = 7
recall: 36747/36750
positive sample: 92982/735000  = 0.1265
negative sample: 642018/735000 = 0.8734